In [ ]:
from IPython.display import display
import arcgis
from arcgis.gis import GIS

gis = GIS(url,username,password,verify_cert=False)

In [ ]:
import os
import json
from arcgis.mapping import WebMap
webmap = gis.content.search(query = 'title: TWE NTP POD Web Map - Richfield Field Office',item_type="Web Map")[0]
webmap.zoomLayer

In [ ]:
from IPython.display import display
import arcgis
from arcgis.gis import GIS
import os
import json
import copy
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer
gis = GIS(url,username,password,verify_cert=False)

webMap = gis.content.search(query='title: TWE NTP POD Web Map - Richfield Field Office',item_type="Web Map")
targetMap = webMap[0].get_data(try_json=True)
webMapJsonCopy = copy.deepcopy(targetMap)
i = 0
for x in webMapJsonCopy['operationalLayers']:
    if webMapJsonCopy['operationalLayers'][i]['title']=='Wildlife Requirement':
        index = i
        wildlife = x
        break
    i+=1
w = webMapJsonCopy['operationalLayers'][index]['layerDefinition']['drawingInfo']['renderer']['uniqueValueInfos']

      
# targetMap['operationalLayers'][30]['layerDefinition']['drawingInfo']['renderer']['uniqueValueInfos'][0]['value']

webmap = WebMap(webMap[0])

ls = [layer for layer in webmap.layers if layer['title']=="Wildlife Requirement"]
wildlife = ls[0]
url = wildlife['url']
layer=FeatureLayer(url)
result = layer.query(where="fieldOffice = 'Richfield Field Office'",out_fields="req_num")
tempList = result.sdf.values.tolist()
reqnum = []
for temp in tempList:
    reqnum.append(temp[1])
print(reqnum)

ls = [x for x in wildlife['layerDefinition']['drawingInfo']['renderer']['uniqueValueInfos']]

appendLs = []
for l in ls:
    content = l
    keyId = content["value"]
    if keyId in reqnum:
        appendLs.append(content)
gis = GIS(url,username,password,verify_cert=False)

webMapJsonCopy['operationalLayers'][index]['layerDefinition']['drawingInfo']['renderer']['uniqueValueInfos']=appendLs
item_properties = {"text":json.dumps(webMapJsonCopy)}
gis = GIS(url,username,password,verify_cert=False)
webMap[0].update(item_properties=item_properties)
# webmap.update()

In [ ]:
masterMap = WebMap(webmap)
masterMap

In [ ]:

masterMap = gis.content.search(query = 'title: TWE NTP POD Web Map - Master')[0]
webmapDict = masterMap.get_data(try_json=True)
ls = ["White River","Caliente","Southern Nevada","Cedar City","Richfield","Vernal","Salt Lake","Fillmore"]
for l in ls:
    title = f"TWE NTP POD Web Map - {l} Field Office"
    snippet = f"This web map contains all the resource and constraints in {l} Field Office"
    tags = ['TWE','POD','resource','constraints',l]
    webmapProperties = {
        'title': title,
        'type': 'Web Map',
        'snippet': snippet,
        'tags': tags,
        'text':json.dumps(webmapDict)
    }
    webmapItem = gis.content.add(webmapProperties)

In [ ]:
from IPython.display import display
import arcgis
from arcgis.gis import GIS
import os
import json
import copy
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

gis = GIS(url,username,password,verify_cert=False)
webMaps = gis.content.search(query = 'title: TWE NTP POD Web Map - ',item_type="Web Map")
titleLs = [webMap.title for webMap in webMaps]
for webMap in webMaps:
#     webMapJson = webMap.get_data(try_json=True)
#     webMapJsonCopy = copy.deepcopy(webMapJson)
    title = webMap.title
    targetMap = WebMap(webMap)

    if title == "TWE NTP POD Web Map - BLM Little Snake Field Office" or title == "TWE NTP POD Web Map - Master" or title == "TWE NTP POD Web Map - BLM Rawlins Field Office":
        pass
    else:
        
#         print(title)
        removeLayers = ['Structures', 'Project Facilities', 'Transmission Centerline', 'Access Roads', 'ROW', 'Work Areas','Land Ownership', 'Boundaries']
        #         layers = [[layer['title'],layer['url']] for layer in targetMap.layers if layer['title'] not in removeLayers]
        layers = [layer for layer in targetMap.layers if layer['title'] not in removeLayers]

        for layer in layers:
            gis = GIS("https://webmap1.swca.com/portal","swcacorp\hitomi.nakamura","Ms4984419",verify_cert=False)
            layerName = layer['title']
            layerUrl = FeatureLayer(layer['url'])
            featureLayer = FeatureLayer(layerUrl)
            fieldOffice = title[22:]
#             fieldsLs = [field['name'] for field in layerUrl.properties.fields]
            if "fieldoffice" in fieldsLs:
                sql = f"fieldoffice = '{fieldOffice}'"
            elif "fieldOffice" in fieldsLs:
                sql = f"fieldOffice = '{fieldOffice}'"
            else:
                pass
#             fields = ", ".join(fieldsLs)
           

            query = layerUrl.query(where=sql,return_count_only=True)

            if int(query) == 0:
#                 print("remove")
                targetMap.remove_layer(layer)
            else:
#                 print("filter")
                del layer['definition_expression']
                layer['layerDefinition']={'definitionExpression':sql}
               
        targetMap.update()
